In [ ]:
!pip install --quiet piexif

In [ ]:
from PIL import Image
import piexif

codec = 'ISO-8859-1'


def merge_gps_dms(gps_data):
    def convert(gps_data):
        degrees = gps_data[0][0] / gps_data[0][1]
        minutes = gps_data[1][0] / gps_data[1][1] / 60.0
        seconds = gps_data[2][0] / gps_data[2][1] / 3600.0
        return degrees + minutes + seconds

    latitude = convert(gps_data['GPSLatitude'])
    longitude = convert(gps_data['GPSLongitude'])

    if 'GPSLatitudeRef' in gps_data and gps_data['GPSLatitudeRef'] == 'S':
        latitude = -latitude
    if 'GPSLongitudeRef' in gps_data and gps_data['GPSLongitudeRef'] == 'W':
        longitude = -longitude

    return latitude, longitude

def exif_to_tag(exif_dict):
    exif_tag_dict = {}
    thumbnail = exif_dict.pop('thumbnail')
    exif_tag_dict['thumbnail'] = thumbnail.decode(codec)

    for ifd in exif_dict:
        exif_tag_dict[ifd] = {}
        for tag in exif_dict[ifd]:
            try:
                element = exif_dict[ifd][tag].decode(codec)

            except AttributeError:
                element = exif_dict[ifd][tag]

            exif_tag_dict[ifd][piexif.TAGS[ifd][tag]["name"]] = element

    return exif_tag_dict


def main():

    filename = 'filename'

    im = Image.open(filename)

    exif_dict = piexif.load(im.info.get('exif'))
    exif_dict = exif_to_tag(exif_dict)

    latitude, longitude = merge_gps_dms(exif_dict['GPS'])

    if latitude is not None and longitude is not None:
      print(f"Latitude: {latitude}")
      print(f"Longitude: {longitude}")
    else:
      print("Error: GPS data conversion failed.")

if __name__ == '__main__':
   main()